# Understanding Containerization
### Docker Containers: Fundamentals, Security, and Hands-On Labs

## Prerequisites:

- Basic understanding of command-line interfaces (Linux/bash or Windows PowerShell preferred)
- Basic familiarity with virtualization concepts (desirable)

## Software:

- Docker Desktop installed (or another compatible Docker environment)

## Agenda

1. Introduction & Core Concepts
    - Lab 1: Basic Container Operations
3. Building Images and Dockerfiles
    - Lab 2: Creating a Dockerfile 
4. Docker Security 
    - Image Security
    - Container Runtime Security
    - Demo: Compromise Emulation
5. Networking and Storage
    - Port Mapping
    - Docker Volumes
    - Lab 3: Communicating Containers
6. Conclusions and what's next






## Lab 1 - Basic Container Operations
###  See Docker CLI cheat sheet here: https://docs.docker.com/get-started/docker_cheatsheet.pdf

1. Pulling an Image

    -  Open a terminal or command prompt.

    -  Run the command: `docker pull nginx`

2. Running a Container

    - Run the command: `docker run -d -p 80:80 nginx`

    - **Explanation:** 

        - `-d` runs the container in detached mode (background)

        - `-p` 80:80 maps port 80 of the container to port 80 on your host machine.

3. Verifying the Container

    -   Open a web browser and go to `http://localhost`. You should see the Nginx welcome page.

    -   In your terminal, type: `docker ps` to list running containers.

4. Stopping and Removing

    - Get the container ID from `docker ps`.

    - Stop the container: docker stop <container_id>
    
    - Remove the container: docker rm <container_id>

    - Bonus command-line tricks for cleaning up unused containers and freeing up disk space:

        - `docker ps -a -q`: This command lists all containers, including stopped ones (-a flag), and extracts only their container IDs (-q flag). It provides a list of container IDs that have exited or are currently stopped.
        
        - `docker stop $(docker ps -a -q)`: Stops all running containers
        
        - `docker rm $(docker ps -a -q)`: Removes all containers